# Analyse et visualisation de données avec Python
## Automatisation du traitement des données
Questions
* Comment automatiser le traitement des données dans Python?
* Comment structurer le traitement avec des fonctions?

Objectifs
* Utiliser des boucles `for` pour automatiser l'analyse de données.
* Générer des fichiers avec des noms uniques.
* Construire du code réutilisable en Python.
* Écrire des fonctions avec des conditions (`if`, `then`, `else`).

## Charger nos données

In [ ]:
# Charger le module pandas
import pandas as pd

# Charger les données
surveys_df = pd.read_csv("../data/surveys.csv")
species_df = pd.read_csv("../data/species.csv")

## Automatiser l'analyse de données avec des boucles `for`

In [ ]:
import os

In [ ]:
dossier_annees = "fichiers_par_annee"
os.mkdir(dossier_annees)

In [ ]:
os.listdir('.')

In [ ]:
for annee in surveys_df['year'].unique():
    # Créer un nom de fichier unique pour chaque année
    nom_fichier = os.path.join(dossier_annees, f"surveys_{annee}.csv")
    print(nom_fichier)

    # Sauvegarder les données complètes de l'année en cours
    surveys_annee = surveys_df[surveys_df['year'] == annee].dropna()
    surveys_annee.to_csv(nom_fichier, index=False)

os.listdir(dossier_annees)

### Exercice - Création de fichiers différents
Au lieu de diviser les données par année, nous voudrions les diviser
par nom complet d'espèce.
Il faut donc au préalable joindre les deux dataframes :
* On veut le détail des espèces à droite dans le DataFrame final
* La jonction doit garder toutes les entrées de `surveys_df`; les
  espèces inconnues auront donc un nom `NaN`

In [ ]:
dossier_especes = "fichiers_especes"
os.mkdir(dossier_especes)  # Créer le dossier

In [ ]:
jonc_gauche = pd.merge(left=surveys_df, right=species_df,
                       how='left', on='species_id')
jonc_gauche

In [ ]:
for espece in jonc_gauche['species'].unique():
    # Créer un nom de fichier unique pour chaque espèce
    nom_fichier = os.path.join(dossier_especes, f"surveys_{espece}.csv")
    print(nom_fichier)

    # Sélectionner les données de l'espèce en cours
    jonc_gauche_espece = jonc_gauche[jonc_gauche['species'] == espece]
    jonc_gauche_espece.to_csv(nom_fichier, index=False)

## Développer du code modulaire et réutilisable avec des fonctions
Dans cet exemple, on doit tout d'abord créer une fonction qui servira
à sauvegarder les données d'une seule année. En argument, il faudra
spécifier :
* Le nom d'un dossier existant
* Le préfixe du nom de fichier CSV
* L'année à sélectionner

In [ ]:
def sauvegarder_annee(donnees, dossier_cible, prefixe, cette_annee):
    """
    Écrire un fichier CSV avec les données d'une seule année.

    donnees --- le DataFrame avec une colonne 'year'
    dossier_cible --- dossier dans lequel sauvegarder le fichier
    prefixe --- préfixe au nom de fichier CSV
    cette_annee --- année à sélectionner
    """

    # Créer un nom de fichier unique pour chaque année
    nom_fichier = os.path.join(dossier_cible, f"{prefixe}{cette_annee}.csv")

    # Sauvegarder les données complètes de l'année en cours
    surveys_annee = donnees[donnees['year'] == cette_annee].dropna()
    surveys_annee.to_csv(nom_fichier, index=False)

In [ ]:
os.mkdir("test")

In [ ]:
sauvegarder_annee(surveys_df, "test", "préfixe_", 1999)

On souhaite ensuite créer une fonction principale qui utilise la
fonction précédente et qui gère les arguments en entrée :
* Créer le dossier automatiquement, mais en testant son existence avant
* Les arguments `annee_debut` et `annee_fin` ont `None` par défaut
  * Trouver la première année si `annee_debut` est `None`
  * Trouver la dernière année si `annee_fin` est `None`
* Boucler sur toutes les années de l'intervalle validé

In [ ]:
def les_annees_en_csv(donnees, dossier_cible, prefixe,
                      annee_debut = None, annee_fin = None):
    """
    Écrire des fichiers CSV avec les données de plusieurs années.

    donnees --- le DataFrame
    dossier_cible --- dossier dans lequel sauvegarder le fichier
    prefixe --- préfixe au nom de fichier CSV
    annee_debut --- première année --- défaut: None - depuis le tout début
    annee_fin --- dernière année --- défaut: None - jusqu'à la toute fin
    """

    if dossier_cible in os.listdir('.'):
        print(f'Le dossier "{dossier_cible}" existe')
    else:
        os.mkdir(dossier_cible)
        print(f'Nouveau dossier: "{dossier_cible}"')

    if not annee_debut:
        annee_debut = donnees['year'].min()

    if not annee_fin:
        annee_fin = donnees['year'].max()

    # "annee_fin" est la dernière année à traiter, donc il faut annee_fin + 1
    for annee in range(annee_debut, annee_fin + 1):
        sauvegarder_annee(donnees, dossier_cible, prefixe, annee)

In [ ]:
les_annees_en_csv(surveys_df, "final", "resultats_", 1995, 1998)

In [ ]:
os.listdir("final")

In [ ]:
les_annees_en_csv(surveys_df, "final", "resultats_")

## Résumé technique
* **Utilisation d'une boucle** pour traiter plusieurs fichiers
* **Définir des fonctions** permet d'alléger le code dans une boucle
* **Utilisation de conditions** pour :
  * tester les paramètres d'une fonction;
  * prendre des décisions selon différents critères.